In [1]:
import pandas as pd
import numpy as np
from skyfield.api import load, Topos, Star, utc
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

import os
import pathlib
import stars_utils

MODEL_DIR = 'models'
MODELS_LIS = ['mm0','mm1','mm2']

In [2]:
MODEL_DIR = 'models'
MODELS_LIS = ['mm0','mm1','mm2']

models = stars_utils.models_loader(MODEL_DIR,MODELS_LIS)

Loading models from directory: models
Attempting to load: models/mars_position_predictor_mm0.keras
Attempting to load: models/mars_position_predictor_mm1.keras
File not found at models/mars_position_predictor_mm2.keras. Falling back to: models/mars_position_predictor_mm2.h5
Successfully loaded 3 models.


In [2]:
# Load the JPL ephemeris data file
planets = load('de421.bsp')
ts = load.timescale()
mars = planets['mars']
earth = planets['earth']

# 2. Define Time Range (e.g., 50 years of data, 1970 to 2020)
start_date = datetime(1970, 1, 1)
end_date = datetime(2025, 1, 1)
time_step = timedelta(days=1)  # Data point every week

# Generate a list of dates
dates = []
current_date = start_date
while current_date <= end_date:
    dates.append(current_date)
    current_date += time_step


In [3]:

# Convert Python datetimes to Skyfield time objects
timezone_aware_dates = [d.replace(tzinfo=utc) for d in dates]
t = ts.utc(timezone_aware_dates)

# 3. Calculate Geocentric Position (Position as seen from Earth)
# astrometric_position is the calculation of a body's position in space
# as seen from a specific location (Earth).
astrometric = earth.at(t).observe(mars)

# Get Right Ascension and Declination
# These are your primary target variables for prediction!
ra, dec, distance = astrometric.radec()

# 4. Create the DataFrame (Your Dataset)
data = {
    'Time_UTC': dates,
    'Julian_Date': t.tdb,
    'RA_deg': ra.degrees,      # Right Ascension (Target Variable)
    'Dec_deg': dec.degrees     # Declination (Target Variable)
}

df = pd.DataFrame(data)

# 5. Save the Dataset
df.to_csv('mars_ephemeris_data.csv', index=False)
print("Dataset created successfully with", len(df), "data points.")

mars_sf_df = df.copy()

Dataset created successfully with 20090 data points.


In [5]:
# # read the jpl file, rename and clean the colums, and remove redundants

# mars_jpl_df = pd.read_csv("/Users/yasha/Desktop/projects/stars/mars_1970_2025.csv", skiprows=7)
# mars_jpl_df = mars_jpl_df.rename(columns={mars_jpl_df.columns[0]: 'Date__(UT)__HR:MN'})
# mars_jpl_df.columns = [col.strip().replace('\\', '') for col in mars_jpl_df.columns]
# cols_to_drop = ['', '.1', '/r']
# mars_jpl_df = mars_jpl_df.drop(columns=cols_to_drop, errors='ignore')
# mars_jpl_df.head()

In [4]:
# Get JD_min as the reference point
JD_min = mars_sf_df.Julian_Date.min()
mars_sf_df["Time_Index"] = mars_sf_df.Julian_Date - JD_min
time_index = mars_sf_df["Time_Index"]

# Add features to the dataframe
PERIOD_YEAR = 365.25
PERIOD_SYNODIC = 780.0

# Earth's Annual Cycle Features
mars_sf_df["Sin_Year"] = np.sin(2*np.pi*time_index/PERIOD_YEAR)
mars_sf_df["Cos_Year"] = np.cos(2*np.pi*time_index/PERIOD_YEAR)

# Mars-Earth Synodic Cycle Features
mars_sf_df["Sin_Synodic"] = np.sin(2*np.pi*time_index/PERIOD_SYNODIC)
mars_sf_df["Cos_Synodic"] = np.cos(2*np.pi*time_index/PERIOD_SYNODIC)


In [5]:
mars_sf_df.shape

(20090, 9)

In [6]:
# Define features (x) and targets (y) for the model 
FEATURES = [
    'Time_Index',
    'Sin_Year', 'Cos_Year',
    'Sin_Synodic', 'Cos_Synodic'
]
TARGETS = ['RA_deg', 'Dec_deg']

X = mars_sf_df[FEATURES]
y = mars_sf_df[TARGETS]

# 80/20 split of the data
split_point = int(len(mars_sf_df)*0.8)

X_train = X[:split_point]
X_test = X[split_point:]
y_train = y[:split_point]
y_test = y[split_point:]

print(f"Total data points: {len(X)}")
print(f"Training period ends at index {split_point} (Date: {df.iloc[split_point]['Time_UTC']})")
print(f"Testing period starts at index {split_point} (Date: {df.iloc[split_point]['Time_UTC']})")


Total data points: 20090
Training period ends at index 16072 (Date: 2014-01-02 00:00:00)
Testing period starts at index 16072 (Date: 2014-01-02 00:00:00)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# RF model training
# - 100 trees, depth of each tree is 15
model = RandomForestRegressor(
    n_estimators=1000,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

# Train the model
print("\nStarting model training...")
model.fit(X_train, y_train)
print("Model training complete.")


Starting model training...
Model training complete.


In [9]:
# Make predictions on the unseen test data
y_pred = model.predict(X_test)

# Calculate RMSE for each target Right Ascension and Declination
rmse_ra = np.sqrt(mean_squared_error(y_test['RA_deg'], y_pred[:, 0]))
rmse_dec = np.sqrt(mean_squared_error(y_test['Dec_deg'], y_pred[:, 1]))

print("\n--- Model Evaluation (Test Set) ---")
print(f"RA (Right Ascension) RMSE: {rmse_ra:.6f} degrees")
print(f"Dec (Declination) RMSE: {rmse_dec:.6f} degrees")

# Calculate the average error distance (Hypotenuse of the error)
total_error = np.sqrt(
    (y_test['RA_deg'] - y_pred[:, 0])**2 + 
    (y_test['Dec_deg'] - y_pred[:, 1])**2
)
print(f"Mean Angular Separation Error: {total_error.mean():.6f} degrees")


--- Model Evaluation (Test Set) ---
RA (Right Ascension) RMSE: 92.835463 degrees
Dec (Declination) RMSE: 13.216534 degrees
Mean Angular Separation Error: 65.860966 degrees


In [10]:
### this does not work well :(

In [8]:
# get the astronomic xyz coordinates of mars from earth's pov
astrometric = earth.at(t).observe(mars)
pos_vector_au = astrometric.xyz.au #pos_vector_au is a 3xN NumPy array, where N is the number of time steps (t).

data = {
    'Time_UTC': dates,
    'Julian_Date': t.tdb,
    
    # Extract the X, Y, and Z components
    'X_au': pos_vector_au[0],  # X-coordinate
    'Y_au': pos_vector_au[1],  # Y-coordinate
    'Z_au': pos_vector_au[2]   # Z-coordinate
}

mars_sf_df = pd.DataFrame(data)

In [9]:
JD_min = mars_sf_df.Julian_Date.min()
mars_sf_df["Time_Index"] = mars_sf_df.Julian_Date - JD_min
time_index = mars_sf_df["Time_Index"]

mars_sf_df["Time_Index_2"] = time_index ** 2
mars_sf_df["Time_Index_3"] = time_index ** 3

# Add features to the dataframe
PERIOD_YEAR = 365.25
PERIOD_MARS = 686.98
PERIOD_SYNODIC = 780.0
PERIOD_JUPITER = 4332.6 # mars cycles are influenced by jupiters movement. added this for fixing.

# Earth's Annual Cycle Features
mars_sf_df["Sin_Year"] = np.sin(2*np.pi*time_index/PERIOD_YEAR)
mars_sf_df["Cos_Year"] = np.cos(2*np.pi*time_index/PERIOD_YEAR)

# Mars's Orbital Cycle Features (NEW and CRUCIAL)
mars_sf_df['Sin_Mars'] = np.sin(2 * np.pi * time_index / PERIOD_MARS)
mars_sf_df['Cos_Mars'] = np.cos(2 * np.pi * time_index / PERIOD_MARS)

# Mars-Earth Synodic Cycle Features
mars_sf_df["Sin_Synodic"] = np.sin(2*np.pi*time_index/PERIOD_SYNODIC)
mars_sf_df["Cos_Synodic"] = np.cos(2*np.pi*time_index/PERIOD_SYNODIC)

# Jupiter's Annual Cycle Features
mars_sf_df['Sin_Jupiter'] = np.sin(2 * np.pi * time_index / PERIOD_JUPITER)
mars_sf_df['Cos_Jupiter'] = np.cos(2 * np.pi * time_index / PERIOD_JUPITER)

mars_sf_df['Sin_Year_Sin_Synodic'] = mars_sf_df['Sin_Year'] * mars_sf_df['Sin_Synodic']
mars_sf_df['Sin_Year_Cos_Synodic'] = mars_sf_df['Sin_Year'] * mars_sf_df['Cos_Synodic']
mars_sf_df['Cos_Year_Sin_Synodic'] = mars_sf_df['Cos_Year'] * mars_sf_df['Sin_Synodic']
mars_sf_df['Cos_Year_Cos_Synodic'] = mars_sf_df['Cos_Year'] * mars_sf_df['Cos_Synodic']

# Define features (x) and targets (y) for the model 
FEATURES = [
    'Time_Index', 'Time_Index_2', 'Time_Index_3', 
    'Sin_Year', 'Cos_Year',
    'Sin_Mars', 'Cos_Mars',
    'Sin_Synodic', 'Cos_Synodic',
]
TARGETS = ['X_au', 'Y_au', 'Z_au']

FEATURES_XGB = [
    'Time_Index', 'Time_Index_2', 'Time_Index_3', 
    'Sin_Year', 'Cos_Year',
    'Sin_Mars', 'Cos_Mars',
    'Sin_Synodic', 'Cos_Synodic',
    'Sin_Year_Sin_Synodic', 'Sin_Year_Cos_Synodic','Cos_Year_Sin_Synodic','Cos_Year_Cos_Synodic'
]
TARGETS = ['X_au', 'Y_au', 'Z_au']


In [10]:
X = mars_sf_df[FEATURES]
X_xgb = mars_sf_df[FEATURES_XGB]
y = mars_sf_df[TARGETS]

# 80/20 split of the data
split_point = int(len(mars_sf_df)*0.8)

X_train = X[:split_point]
X_test = X[split_point:]

X_xgb_train = X_xgb[:split_point]
X_xgb_test = X_xgb[split_point:]

y_train = y[:split_point]
y_test = y[split_point:]

# apply standart scalar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_xgb_train_scaled = scaler.fit_transform(X_xgb_train)
X_xgb_test_scaled = scaler.transform(X_xgb_test)

# Fit and transform the training targets
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)

# Transform the testing targets
y_test_scaled = y_scaler.transform(y_test)

print(f"Total data points: {len(X)}")
print(f"Training period ends at index {split_point} (Date: {df.iloc[split_point]['Time_UTC']})")
print(f"Testing period starts at index {split_point} (Date: {df.iloc[split_point]['Time_UTC']})")

Total data points: 20090
Training period ends at index 16072 (Date: 2014-01-02 00:00:00)
Testing period starts at index 16072 (Date: 2014-01-02 00:00:00)


In [15]:
# RF model training
# - 150 trees, depth of each tree is 15
model = RandomForestRegressor(
    n_estimators=150,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

# Train the model
print("\nStarting model training...")
model.fit(X_train, y_train)
print("Model training complete.")


Starting model training...
Model training complete.


In [16]:
# Make predictions on the unseen test data
y_pred = model.predict(X_test)

# Convert the NumPy prediction array back to a DataFrame for easy calculation
y_pred_df = pd.DataFrame(y_pred, columns=TARGETS, index=y_test.index)

# Calculate RMSE for each target coordinate
rmse_x = np.sqrt(mean_squared_error(y_test['X_au'], y_pred_df['X_au']))
rmse_y = np.sqrt(mean_squared_error(y_test['Y_au'], y_pred_df['Y_au']))
rmse_z = np.sqrt(mean_squared_error(y_test['Z_au'], y_pred_df['Z_au']))

print("\n--- Model Evaluation (Test Set) ---")
print(f"X_au RMSE: {rmse_x:.6f} AU")
print(f"Y_au RMSE: {rmse_y:.6f} AU")
print(f"Z_au RMSE: {rmse_z:.6f} AU")



--- Model Evaluation (Test Set) ---
X_au RMSE: 0.056263 AU
Y_au RMSE: 0.049758 AU
Z_au RMSE: 0.022668 AU


In [17]:
# MLP NN Implemantation

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# use early stopping for when there is no improvement for 150 epochs
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=150, 
    restore_best_weights=True
)

# Reduce the LR by 50% (factor=0.5), If no improvement for 50 epochs, drop the LR, Don't let the LR drop below 1e-7 (0.0000001)
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,       
    patience=50,      
    min_lr=1e-7       
)

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 
regularizer_strength = 0.001 

model = Sequential([
    # Apply kernel_regularizer to hidden layers
    Dense(128, activation='relu', kernel_regularizer=l2(0.0001), 
          input_shape=(X_train_scaled.shape[1],)), 
    
    Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.0001)), 
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),

    Dense(3, activation='linear') 
])

model.compile(optimizer=custom_optimizer, loss='mse') 

# use 10000 spochs with batch size of 32
print("\nStarting Neural Network training (100 epochs)...")

history = model.fit(
    X_train_scaled, 
    y_train_scaled, 
    epochs=10000,             
    batch_size=32,          
    validation_data=(X_test_scaled, y_test_scaled),
    callbacks=[early_stopping, lr_scheduler],
    verbose=1              
)

print("Model training complete.")


Starting Neural Network training (100 epochs)...
Epoch 1/10000


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step - loss: 0.0532 - val_loss: 0.0392 - learning_rate: 0.0010
Epoch 2/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 0.0209 - val_loss: 0.0257 - learning_rate: 0.0010
Epoch 3/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - loss: 0.0129 - val_loss: 0.0178 - learning_rate: 0.0010
Epoch 4/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 0.0084 - val_loss: 0.0131 - learning_rate: 0.0010
Epoch 5/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - loss: 0.0056 - val_loss: 0.0149 - learning_rate: 0.0010
Epoch 6/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - loss: 0.0043 - val_loss: 0.0077 - learning_rate: 0.0010
Epoch 7/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 0.0032 - val_loss: 0.0087 - learning_rate: 0.0010
Epoch 8/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - loss: 0.0028 - val_loss: 0.0044 - learning_rate: 0.0010
Epoch 9/10000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: 0.0022 - val_loss: 0.0041 - l

KeyboardInterrupt: 

In [12]:
# Predict on the test set
y_pred_scaled = model.predict(X_test_scaled)

# get inverse value (becuase of the scaling)
y_pred_mlp_au = y_scaler.inverse_transform(y_pred_scaled)

# Calculate the loss (MSE) on the test set
y_test_np = y_test.values
loss_mse = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
rmse_au = np.sqrt(loss_mse) 

# Calculate RMSE for each coordinate individually
rmse_x = np.sqrt(mean_squared_error(y_test_np[:, 0], y_pred_mlp_au[:, 0]))
rmse_y = np.sqrt(mean_squared_error(y_test_np[:, 1], y_pred_mlp_au[:, 1]))
rmse_z = np.sqrt(mean_squared_error(y_test_np[:, 2], y_pred_mlp_au[:, 2]))

print("\n--- Model Evaluation (Neural Network Test Set) ---")
print(f"Overall Averaged RMSE: {rmse_au:.6f} AU")
print(f"X-coordinate RMSE: {rmse_x:.6f} AU")
print(f"Y-coordinate RMSE: {rmse_y:.6f} AU")
print(f"Z-coordinate RMSE: {rmse_z:.6f} AU")

126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step

--- Model Evaluation (Neural Network Test Set) ---
Overall Averaged RMSE: 0.019528 AU
X-coordinate RMSE: 0.003944 AU
Y-coordinate RMSE: 0.003822 AU
Z-coordinate RMSE: 0.001565 AU


In [ ]:
# save the model
MODEL_DIR = 'models'
MODEL_FILENAME = 'mars_position_predictor_final.keras' 

# # Create the directory if it doesn't exist
# os.makedirs(MODEL_DIR, exist_ok=True) 

# Save the model
model.save(os.path.join(MODEL_DIR, MODEL_FILENAME))

print(f"Model saved successfully to {os.path.join(MODEL_DIR, MODEL_FILENAME)}")

Model saved successfully to models/mars_position_predictor_final.keras


In [11]:
# Load the model
model_path = os.path.join(MODEL_DIR, MODEL_FILENAME)
model = tf.keras.models.load_model(model_path)

In [13]:

# Run the conversion function
radec_predictions = stars_utils.xyz_to_radec(y_pred_mlp_au)

print("--- Predicted RA/Dec Coordinates (Degrees) ---")
print(radec_predictions.head())

--- Predicted RA/Dec Coordinates (Degrees) ---
   Predicted_RA_deg  Predicted_Dec_deg
0        191.896774          -2.759812
1        192.311584          -2.923764
2        192.723251          -3.086056
3        193.131683          -3.246643
4        193.522125          -3.398860


In [35]:
import xgboost as xgb

xgb_base = {
    'n_estimators': 5000,
    'learning_rate': 0.01,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'n_jobs': -1,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

num_boost_round = 5000 # Max number of trees

y_train_cols = [y_train.values[:, i] for i in range(3)] # [y_train_X, y_train_Y, y_train_Z]
y_test_cols = [y_test.values[:, i] for i in range(3)]   # [y_test_X, y_test_Y, y_test_Z]

dtrain_cols = [xgb.DMatrix(X_xgb_train_scaled, label=y) for y in y_train_cols]
dtest_cols = [xgb.DMatrix(X_xgb_test_scaled, label=y) for y in y_test_cols]

# Initialize prediction array
N_test = X_xgb_test_scaled.shape[0]
y_pred_xgb_tuned_au = np.zeros((N_test, 3))

coord_names = ['X', 'Y', 'Z']

print("Starting manual, stable XGBoost training (one model per coordinate)...")
for i in range(3):
    print(f"\n--- Training {coord_names[i]}-coordinate model ---")
    
    # Fit the single-output model with EARLY STOPPING
    model = xgb.train(
        params=xgb_base,
        dtrain=dtrain_cols[i],
        num_boost_round=num_boost_round,
        
        # This is the correct native way to pass the eval set
        evals=[(dtest_cols[i], 'validation')], 
        
        # This is the correct native way to pass early stopping
        early_stopping_rounds=100, 
        
        # verbose=False,
    )
    
    # Store predictions
    y_pred_xgb_tuned_au[:, i] = model.predict(dtest_cols[i], iteration_range=(0, model.best_iteration))
    
    print(f"Model for {coord_names[i]} stopped at best iteration: {model.best_iteration}")


print(f"Prediction output shape: {y_pred_xgb_tuned_au.shape}")
print(f"Expected shape: ({len(X_xgb_test_scaled)}, 3)")


Starting manual, stable XGBoost training (one model per coordinate)...

--- Training X-coordinate model ---
[0]	validation-rmse:1.24974
[1]	validation-rmse:1.23953
[2]	validation-rmse:1.22772
[3]	validation-rmse:1.21610
[4]	validation-rmse:1.20448
[5]	validation-rmse:1.19381
[6]	validation-rmse:1.18232
[7]	validation-rmse:1.17092
[8]	validation-rmse:1.15985
[9]	validation-rmse:1.14877
[10]	validation-rmse:1.13795
[11]	validation-rmse:1.12961
[12]	validation-rmse:1.12006
[13]	validation-rmse:1.10950
[14]	validation-rmse:1.09905
[15]	validation-rmse:1.08860
[16]	validation-rmse:1.07836
[17]	validation-rmse:1.06805
[18]	validation-rmse:1.05799
[19]	validation-rmse:1.04800
[20]	validation-rmse:1.03799
[21]	validation-rmse:1.02804
[22]	validation-rmse:1.01824
[23]	validation-rmse:1.00845
[24]	validation-rmse:0.99871
[25]	validation-rmse:0.98930
[26]	validation-rmse:0.98012
[27]	validation-rmse:0.97092
[28]	validation-rmse:0.96190
[29]	validation-rmse:0.95289
[30]	validation-rmse:0.94393
[31

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [16:31:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


[110]	validation-rmse:0.44600
[111]	validation-rmse:0.44183
[112]	validation-rmse:0.43774
[113]	validation-rmse:0.43374
[114]	validation-rmse:0.42968
[115]	validation-rmse:0.42584
[116]	validation-rmse:0.42187
[117]	validation-rmse:0.41806
[118]	validation-rmse:0.41422
[119]	validation-rmse:0.41029
[120]	validation-rmse:0.40655
[121]	validation-rmse:0.40305
[122]	validation-rmse:0.39943
[123]	validation-rmse:0.39571
[124]	validation-rmse:0.39238
[125]	validation-rmse:0.38876
[126]	validation-rmse:0.38517
[127]	validation-rmse:0.38162
[128]	validation-rmse:0.37816
[129]	validation-rmse:0.37465
[130]	validation-rmse:0.37115
[131]	validation-rmse:0.36779
[132]	validation-rmse:0.36443
[133]	validation-rmse:0.36104
[134]	validation-rmse:0.35769
[135]	validation-rmse:0.35438
[136]	validation-rmse:0.35111
[137]	validation-rmse:0.34825
[138]	validation-rmse:0.34512
[139]	validation-rmse:0.34191
[140]	validation-rmse:0.33873
[141]	validation-rmse:0.33564
[142]	validation-rmse:0.33254
[143]	vali

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [16:32:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


[99]	validation-rmse:0.44812
[100]	validation-rmse:0.44399
[101]	validation-rmse:0.43991
[102]	validation-rmse:0.43569
[103]	validation-rmse:0.43170
[104]	validation-rmse:0.42795
[105]	validation-rmse:0.42397
[106]	validation-rmse:0.41997
[107]	validation-rmse:0.41600
[108]	validation-rmse:0.41206
[109]	validation-rmse:0.40825
[110]	validation-rmse:0.40483
[111]	validation-rmse:0.40094
[112]	validation-rmse:0.39718
[113]	validation-rmse:0.39340
[114]	validation-rmse:0.39018
[115]	validation-rmse:0.38655
[116]	validation-rmse:0.38322
[117]	validation-rmse:0.37967
[118]	validation-rmse:0.37605
[119]	validation-rmse:0.37256
[120]	validation-rmse:0.36902
[121]	validation-rmse:0.36548
[122]	validation-rmse:0.36198
[123]	validation-rmse:0.35864
[124]	validation-rmse:0.35520
[125]	validation-rmse:0.35181
[126]	validation-rmse:0.34847
[127]	validation-rmse:0.34518
[128]	validation-rmse:0.34204
[129]	validation-rmse:0.33881
[130]	validation-rmse:0.33562
[131]	validation-rmse:0.33246
[132]	valid

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [16:32:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


[109]	validation-rmse:0.18586
[110]	validation-rmse:0.18433
[111]	validation-rmse:0.18257
[112]	validation-rmse:0.18095
[113]	validation-rmse:0.17925
[114]	validation-rmse:0.17780
[115]	validation-rmse:0.17627
[116]	validation-rmse:0.17482
[117]	validation-rmse:0.17327
[118]	validation-rmse:0.17164
[119]	validation-rmse:0.17009
[120]	validation-rmse:0.16852
[121]	validation-rmse:0.16692
[122]	validation-rmse:0.16535
[123]	validation-rmse:0.16387
[124]	validation-rmse:0.16234
[125]	validation-rmse:0.16082
[126]	validation-rmse:0.15932
[127]	validation-rmse:0.15785
[128]	validation-rmse:0.15645
[129]	validation-rmse:0.15498
[130]	validation-rmse:0.15355
[131]	validation-rmse:0.15212
[132]	validation-rmse:0.15074
[133]	validation-rmse:0.14933
[134]	validation-rmse:0.14794
[135]	validation-rmse:0.14660
[136]	validation-rmse:0.14524
[137]	validation-rmse:0.14401
[138]	validation-rmse:0.14277
[139]	validation-rmse:0.14154
[140]	validation-rmse:0.14027
[141]	validation-rmse:0.13899
[142]	vali

In [36]:
# Calculate RMSE for each coordinate individually
rmse_x_xgb = np.sqrt(mean_squared_error(y_test_np[:, 0], y_pred_xgb_tuned_au[:, 0]))
rmse_y_xgb = np.sqrt(mean_squared_error(y_test_np[:, 1], y_pred_xgb_tuned_au[:, 1]))
rmse_z_xgb = np.sqrt(mean_squared_error(y_test_np[:, 2], y_pred_xgb_tuned_au[:, 2]))

# Calculate the overall averaged RMSE
rmse_overall_ens = np.mean([rmse_x_xgb, rmse_y_xgb, rmse_z_xgb])

print("\n--- Model Evaluation (XGBoost) ---")
print(f"Overall Averaged RMSE: {rmse_overall_ens:.6f} AU")
print(f"AUX-coordinate RMSE: {rmse_x_xgb:.6f} AU")
print(f"AUY-coordinate RMSE: {rmse_y_xgb:.6f} AU")
print(f"AUZ-coordinate RMSE: {rmse_z_xgb:.6f} AU")


--- Model Evaluation (XGBoost) ---
Overall Averaged RMSE: 0.022144 AU
AUX-coordinate RMSE: 0.027297 AU
AUY-coordinate RMSE: 0.022760 AU
AUZ-coordinate RMSE: 0.016376 AU


In [ ]:
# import xgboost as xgb # type: ignore

# xgb_base = {
#     'n_estimators': 5000,
#     'learning_rate': 0.01,
#     'max_depth': 6,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'random_state': 42,
#     'n_jobs': -1,
#     'objective': 'reg:squarederror'
# }

# y_train_cols = [y_train.values[:, i] for i in range(3)] # [y_train_X, y_train_Y, y_train_Z]
# y_test_cols = [y_test.values[:, i] for i in range(3)]   # [y_test_X, y_test_Y, y_test_Z]

# # Initialize prediction array
# N_test = X_test_scaled.shape[0]
# y_pred_xgb_tuned_au = np.zeros((N_test, 3))

# coord_names = ['X', 'Y', 'Z']

# print("Starting manual, stable XGBoost training (one model per coordinate)...")
# for i in range(3):
#     print(f"\n--- Training {coord_names[i]}-coordinate model ---")
    
#     # Initialize a fresh XGBRegressor for each target
#     model = xgb.XGBRegressor(**xgb_base)
    
#     early_stop_callback = xgb.callback.early_stop(
#         rounds=100, # Use 'rounds' instead of 'stopping_rounds' for callback object
#         metric_name='rmse',
#         data_name='validation_0',
#         save_best=True
#     )
    
#     # Fit the single-output model with EARLY STOPPING
#     model.fit(
#         X_train_scaled, 
#         y_train_cols[i], # Single target column
        
#         # Pass the validation set with a named tuple (validation_0)
#         eval_set=[(X_test_scaled, y_test_cols[i], 'validation_0')], 
        
#         # Pass the callback list
#         callbacks=[early_stop_callback],
#         verbose=False,
#     )
    
#     # Store predictions
#     y_pred_xgb_tuned_au[:, i] = model.predict(X_test_scaled)
    
#     print(f"Model for {coord_names[i]} stopped at best iteration: {model.best_iteration}")


# print(f"Prediction output shape: {y_pred_xgb_tuned_au.shape}")
# print(f"Expected shape: ({len(X_test_scaled)}, 3)")


Starting manual, stable XGBoost training (one model per coordinate)...

--- Training X-coordinate model ---


AttributeError: module 'xgboost.callback' has no attribute 'early_stop'

In [40]:
# Running ensamble

# Define Weights
W_MLP = 0.7
W_XGB = 0.3

# Create the Ensemble Prediction (weighted average of the two models)
y_pred_ensemble_au = (W_MLP * y_pred_mlp_au) + (W_XGB * y_pred_xgb_tuned_au)

# Evaluate the Ensemble
# You need your original unscaled test targets (y_test) as a NumPy array for comparison.
# Assuming 'y_test' is your DataFrame of unscaled targets (X_au, Y_au, Z_au)
y_test_np = y_test.values 

# Calculate RMSE for each coordinate individually
rmse_x_ens = np.sqrt(mean_squared_error(y_test_np[:, 0], y_pred_ensemble_au[:, 0]))
rmse_y_ens = np.sqrt(mean_squared_error(y_test_np[:, 1], y_pred_ensemble_au[:, 1]))
rmse_z_ens = np.sqrt(mean_squared_error(y_test_np[:, 2], y_pred_ensemble_au[:, 2]))

# Calculate the overall averaged RMSE
rmse_overall_ens = np.mean([rmse_x_ens, rmse_y_ens, rmse_z_ens])

print("\n--- Model Evaluation (XGBoost + MLP Ensemble) ---")
print(f"Overall Averaged RMSE: {rmse_overall_ens:.6f} AU")
print(f"AUX-coordinate RMSE: {rmse_x_ens:.6f} AU")
print(f"AUY-coordinate RMSE: {rmse_y_ens:.6f} AU")
print(f"AUZ-coordinate RMSE: {rmse_z_ens:.6f} AU")


--- Model Evaluation (XGBoost + MLP Ensemble) ---
Overall Averaged RMSE: 0.006965 AU
AUX-coordinate RMSE: 0.008629 AU
AUY-coordinate RMSE: 0.007312 AU
AUZ-coordinate RMSE: 0.004954 AU


In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# --- MODEL CONFIGURATION ---
NUM_MODELS = 3 # Number of MLP models in the ensemble
# Set a high number of epochs since Early Stopping will handle when to stop
EPOCHS = 5000   
INPUT_SHAPE = X_train_scaled.shape[1] # Number of input features
models = []
y_pred_list = []

# --- Define Callbacks for Training ---
early_stopping_callback = EarlyStopping(
    monitor='val_loss', # Monitors validation loss
    patience=150,        # Stops training after 150 epochs of no improvement
    restore_best_weights=True,
    min_delta = 1e-7 
    )

lr_on_plateau_callback = ReduceLROnPlateau(
    monitor='val_loss', # Monitors validation loss
    factor=0.5,         # Reduces learning rate by 50%
    patience=50,        # If no improvement for 50 epochs, reduce LR
    min_lr=1e-7         # The minimum learning rate to allow
)

callbacks = [early_stopping_callback, lr_on_plateau_callback]

# --- 1. BUILD AND TRAIN THE INDIVIDUAL MLP MODELS ---
print(f"Building and training {NUM_MODELS} diverse MLP models...")

for i in range(NUM_MODELS):
    # Use a different random seed for each model to ensure diverse weight initializations
    tf.keras.utils.set_random_seed(i + 1)
    
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.0001), 
              input_shape=(INPUT_SHAPE,)), 
        
        Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.0001)), 
        
        Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),

        Dense(3, activation='linear') 
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print(f"\n--- Training MLP Model {i+1} ---")
    model.fit(
        X_train_scaled, 
        y_train.values, 
        epochs=EPOCHS, 
        validation_data=(X_test_scaled, y_test.values),
        callbacks=callbacks, # Pass the list of callbacks here
        verbose=1 # Changed to 1 to show the callback messages
    )
    
    models.append(model)
    
    # Generate predictions for the current model and store them
    y_pred = model.predict(X_test_scaled)
    y_pred_list.append(y_pred)
    
    # Calculate and print the individual model's RMSE for comparison
    rmse_x = np.sqrt(mean_squared_error(y_test.values[:, 0], y_pred[:, 0]))
    rmse_y = np.sqrt(mean_squared_error(y_test.values[:, 1], y_pred[:, 1]))
    rmse_z = np.sqrt(mean_squared_error(y_test.values[:, 2], y_pred[:, 2]))
    overall_rmse = np.mean([rmse_x, rmse_y, rmse_z])
    
    print(f"Model {i+1} Test Set RMSE: {overall_rmse:.6f} AU")

# --- 2. CREATE THE ENSEMBLE PREDICTION ---
# The ensemble prediction is the simple average of all individual model predictions.
y_ensemble_pred_au = np.mean(y_pred_list, axis=0)

# --- 3. EVALUATE THE ENSEMBLE PERFORMANCE ---
print("\n--- Final Ensemble Evaluation ---")
rmse_x_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 0], y_ensemble_pred_au[:, 0]))
rmse_y_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 1], y_ensemble_pred_au[:, 1]))
rmse_z_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 2], y_ensemble_pred_au[:, 2]))
overall_ensemble_rmse = np.mean([rmse_x_ensemble, rmse_y_ensemble, rmse_z_ensemble])

print(f"Ensemble X-coordinate RMSE: {rmse_x_ensemble:.6f} AU")
print(f"Ensemble Y-coordinate RMSE: {rmse_y_ensemble:.6f} AU")
print(f"Ensemble Z-coordinate RMSE: {rmse_z_ensemble:.6f} AU")
print(f"Overall Ensemble RMSE: {overall_ensemble_rmse:.6f} AU")

# --- 4. Compare with your best single MLP result ---
# Replace the value below with your actual best MLP RMSE (approx. 0.0031)
BEST_SINGLE_MLP_RMSE = 0.003100
print("\n--- Final Comparison ---")
print(f"Best Single MLP Overall RMSE: {BEST_SINGLE_MLP_RMSE:.6f} AU")
print(f"Multi-MLP Ensemble Overall RMSE: {overall_ensemble_rmse:.6f} AU")

# Determine if the ensemble is better
if overall_ensemble_rmse < BEST_SINGLE_MLP_RMSE:
    print("\nSUCCESS: The multi-model MLP ensemble is more accurate!")
else:
    print("\nRESULT: The single MLP model remains the most accurate.")


Building and training 3 diverse MLP models...

--- Training MLP Model 1 ---
Epoch 1/5000


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 937us/step - loss: 0.0556 - val_loss: 0.0392 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 0.0226 - val_loss: 0.0258 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - loss: 0.0144 - val_loss: 0.0168 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - loss: 0.0093 - val_loss: 0.0109 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - loss: 0.0063 - val_loss: 0.0084 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: 0.0047 - val_loss: 0.0067 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 0.0036 - val_loss: 0.0057 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 0.0031 - val_loss: 0.0058 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: 0.0024 - val_loss: 0.0045 - learning_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 864us/step - loss: 0.0592 - val_loss: 0.0410 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - loss: 0.0218 - val_loss: 0.0235 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 0.0138 - val_loss: 0.0158 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - loss: 0.0092 - val_loss: 0.0117 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - loss: 0.0063 - val_loss: 0.0095 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - loss: 0.0046 - val_loss: 0.0091 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - loss: 0.0039 - val_loss: 0.0092 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - loss: 0.0031 - val_loss: 0.0077 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - loss: 0.0027 - val_loss: 0.0061 - learning_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 859us/step - loss: 0.0525 - val_loss: 0.0503 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.0224 - val_loss: 0.0318 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 0.0141 - val_loss: 0.0203 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - loss: 0.0091 - val_loss: 0.0187 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - loss: 0.0063 - val_loss: 0.0191 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - loss: 0.0047 - val_loss: 0.0075 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.0036 - val_loss: 0.0052 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - loss: 0.0028 - val_loss: 0.0087 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 0.0024 - val_loss: 0.0039 - learning_

In [32]:
MODEL_DIR = 'models'
MODEL_FILENAME = 'mars_position_predictor_mm1.keras' 

# # Create the directory if it doesn't exist
# os.makedirs(MODEL_DIR, exist_ok=True) 

# Save the model
models[0].save(os.path.join(MODEL_DIR, MODEL_FILENAME))

print(f"Model saved successfully to {os.path.join(MODEL_DIR, MODEL_FILENAME)}")

Model saved successfully to models/mars_position_predictor_mm1.keras


In [31]:
y_pred_mm1 = y_pred_list[0]
print("\n--- MM1 Ensemble Evaluation ---")
rmse_x_mm1 = np.sqrt(mean_squared_error(y_test.values[:, 0], y_pred_mm1[:, 0]))
rmse_y_mm1 = np.sqrt(mean_squared_error(y_test.values[:, 1], y_pred_mm1[:, 1]))
rmse_z_mm1 = np.sqrt(mean_squared_error(y_test.values[:, 2], y_pred_mm1[:, 2]))
overall_mm1_rmse = np.mean([rmse_x_mm1, rmse_y_mm1, rmse_z_mm1])

print(f"Ensemble X-coordinate RMSE: {rmse_x_mm1:.6f} AU")
print(f"Ensemble Y-coordinate RMSE: {rmse_y_mm1:.6f} AU")
print(f"Ensemble Z-coordinate RMSE: {rmse_z_mm1:.6f} AU")
print(f"Overall Ensemble RMSE: {overall_mm1_rmse:.6f} AU")


--- MM1 Ensemble Evaluation ---
Ensemble X-coordinate RMSE: 0.003033 AU
Ensemble Y-coordinate RMSE: 0.003884 AU
Ensemble Z-coordinate RMSE: 0.001660 AU
Overall Ensemble RMSE: 0.002859 AU


In [30]:
y_pred_mm1 = y_pred_list[0]
y_pred_list_ens = [y_pred_mlp_au, y_pred_mm1]
y_ensemble_pred_au = np.mean(y_pred_list_ens, axis=0)

# --- 3. EVALUATE THE ENSEMBLE PERFORMANCE ---
print("\n--- Final Ensemble Evaluation ---")
rmse_x_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 0], y_ensemble_pred_au[:, 0]))
rmse_y_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 1], y_ensemble_pred_au[:, 1]))
rmse_z_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 2], y_ensemble_pred_au[:, 2]))
overall_ensemble_rmse = np.mean([rmse_x_ensemble, rmse_y_ensemble, rmse_z_ensemble])

print(f"Ensemble X-coordinate RMSE: {rmse_x_ensemble:.6f} AU")
print(f"Ensemble Y-coordinate RMSE: {rmse_y_ensemble:.6f} AU")
print(f"Ensemble Z-coordinate RMSE: {rmse_z_ensemble:.6f} AU")
print(f"Overall Ensemble RMSE: {overall_ensemble_rmse:.6f} AU")

# --- 4. Compare with your best single MLP result ---
# Replace the value below with your actual best MLP RMSE (approx. 0.0031)
BEST_SINGLE_MLP_RMSE = 0.003100
print("\n--- Final Comparison ---")
print(f"Best Single MLP Overall RMSE: {BEST_SINGLE_MLP_RMSE:.6f} AU")
print(f"Multi-MLP Ensemble Overall RMSE: {overall_ensemble_rmse:.6f} AU")

# Determine if the ensemble is better
if overall_ensemble_rmse < BEST_SINGLE_MLP_RMSE:
    print("\nSUCCESS: The multi-model MLP ensemble is more accurate!")
else:
    print("\nRESULT: The single MLP model remains the most accurate.")


--- Final Ensemble Evaluation ---
Ensemble X-coordinate RMSE: 0.002818 AU
Ensemble Y-coordinate RMSE: 0.003294 AU
Ensemble Z-coordinate RMSE: 0.001294 AU
Overall Ensemble RMSE: 0.002468 AU

--- Final Comparison ---
Best Single MLP Overall RMSE: 0.003100 AU
Multi-MLP Ensemble Overall RMSE: 0.002468 AU

SUCCESS: The multi-model MLP ensemble is more accurate!


In [27]:

# --- CRITICAL FIX REMOVED: tf.config.run_functions_eagerly(True) has been removed.
# This should revert to fast Graph Execution mode.

# --- ASSUME YOUR DATA IS ALREADY LOADED AND PREPROCESSED ---
# X_train_scaled, y_train, X_test_scaled, y_test
# (Assuming your data variables are correctly loaded in the environment)

# --- NEW: OPTIMIZE DATA LOADING FOR PERFORMANCE (FASTEST PATH) ---
# We define raw slices once and optimize the pipeline per model inside the loop.
BATCH_SIZE = 32 # Standard batch size
AUTOTUNE = tf.data.AUTOTUNE 

# 1. Convert NumPy arrays to raw, unshuffled tf.data.Dataset slices (defined once)
raw_train_dataset = Dataset.from_tensor_slices((X_train_scaled, y_train.values.astype(np.float32)))
raw_test_dataset = Dataset.from_tensor_slices((X_test_scaled, y_test.values.astype(np.float32)))


# --- MODEL CONFIGURATION ---
NUM_MODELS = 9 # Assuming you set this to 9 in your environment
# Set a high number of epochs since Early Stopping will handle when to stop
EPOCHS = 5000   
INPUT_SHAPE = X_train_scaled.shape[1] # Number of input features
models = []
y_pred_list = []

# Variables for tracking the best individual model
best_individual_rmse = float('inf')
best_model_index = -1
best_model_seed = -1
best_model_predictions = None
# Replace the value below with your actual best single MLP RMSE (approx. 0.0031)
BEST_SINGLE_MLP_RMSE = 0.003100


# --- Define Callbacks for Training ---
early_stopping_callback = EarlyStopping(
    monitor='val_loss', 
    patience=150,        
    restore_best_weights=True,
    # REVERTED: Removed min_delta to use the default (0.0), which aligns with the original, best-performing setup.
)

lr_on_plateau_callback = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,         
    patience=50,        
    min_lr=1e-7         
)

callbacks = [early_stopping_callback, lr_on_plateau_callback]

# --- 1. BUILD AND TRAIN THE INDIVIDUAL MLP MODELS ---
print(f"Building and training {NUM_MODELS} diverse MLP models...")

for i in range(NUM_MODELS):
    tf.keras.backend.clear_session()
    current_seed = i + 1
    # Set the seed for model weight initialization
    tf.keras.utils.set_random_seed(current_seed)
    
    # --- CRITICAL FIX: Create unique shuffled datasets for each model ---
    # 2. Shuffle, batch, and prefetch for optimal pipeline speed
    # We use the current_seed to ensure each model sees the data in a different order.
    train_dataset = raw_train_dataset.shuffle(buffer_size=1024, seed=current_seed).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    test_dataset = raw_test_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    
    # --- MODEL ARCHITECTURE REVERTED TO ORIGINAL HIGH-PERFORMANCE STRUCTURE ---
    model = Sequential([
        Dense(128, activation='relu', 
              kernel_regularizer=l2(0.0001), # Reverted L2 to original 0.0001
              input_shape=(INPUT_SHAPE,)), 
        
        Dense(256, activation='relu',
              kernel_regularizer=l2(0.0001)), # Reverted L2 to original 0.0001
        
        Dense(128, activation='relu', 
              kernel_regularizer=l2(0.0001)), # Reverted L2 to original 0.0001
        
        Dense(64, activation='relu'),

        Dense(3, activation='linear') 
    ])
    # --- END REVERT ---
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error')
    
    print(f"\n--- Training MLP Model {i+1} (Seed: {current_seed}) ---")
    
    # Use the optimized datasets for fastest training
    model.fit(
        train_dataset, 
        epochs=EPOCHS, 
        validation_data=test_dataset, 
        callbacks=callbacks, 
        verbose=1 
    )
    
    models.append(model)
    
    # Generate predictions using the raw NumPy array (this usually runs fine)
    y_pred = model.predict(X_test_scaled)
    y_pred_list.append(y_pred)
    
    # Calculate and print the individual model's RMSE for comparison
    rmse_x = np.sqrt(mean_squared_error(y_test.values[:, 0], y_pred[:, 0]))
    rmse_y = np.sqrt(mean_squared_error(y_test.values[:, 1], y_pred[:, 1]))
    rmse_z = np.sqrt(mean_squared_error(y_test.values[:, 2], y_pred[:, 2]))
    overall_rmse = np.mean([rmse_x, rmse_y, rmse_z])
    
    print(f"Model {i+1} Test Set RMSE: {overall_rmse:.6f} AU")
    
    if overall_rmse < best_individual_rmse:
        best_individual_rmse = overall_rmse
        best_model_index = i
        best_model_seed = current_seed
        best_model_predictions = y_pred.copy()

# --- 2. CREATE THE ENSEMBLE PREDICTION ---
y_ensemble_pred_au = np.mean(y_pred_list, axis=0)

# --- 3. EVALUATE THE ENSEMBLE PERFORMANCE ---
print("\n--- Final Ensemble Evaluation ---")
rmse_x_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 0], y_ensemble_pred_au[:, 0]))
rmse_y_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 1], y_ensemble_pred_au[:, 1]))
rmse_z_ensemble = np.sqrt(mean_squared_error(y_test.values[:, 2], y_ensemble_pred_au[:, 2]))
overall_ensemble_rmse = np.mean([rmse_x_ensemble, rmse_y_ensemble, rmse_z_ensemble])

print(f"Ensemble X-coordinate RMSE: {rmse_x_ensemble:.6f} AU")
print(f"Ensemble Y-coordinate RMSE: {rmse_y_ensemble:.6f} AU")
print(f"Ensemble Z-coordinate RMSE: {overall_ensemble_rmse:.6f} AU")

# --- 4. Final Comparison and Extraction of Best Model ---
print("\n--- Final Comparison ---")
print(f"Best Single MLP Overall RMSE: {BEST_SINGLE_MLP_RMSE:.6f} AU")
print(f"Multi-MLP Ensemble Overall RMSE: {overall_ensemble_rmse:.6f} AU")

print(f"\n--- Best Individual Model Summary ---")
print(f"Best Individual Model (Index {best_model_index+1}) achieved RMSE: {best_individual_rmse:.6f} AU")
print(f"The random seed used for this best model was: {best_model_seed}")

# --- 5. SAVING THE BEST MODEL ---
if overall_ensemble_rmse < best_individual_rmse and overall_ensemble_rmse < BEST_SINGLE_MLP_RMSE:
    print("\nCONCLUSION: The Multi-MLP Ensemble is the most accurate predictor.")
    MODEL_SAVE_NAME = "final_best_ensemble_predictions.npy"
    np.save(MODEL_SAVE_NAME, y_ensemble_pred_au)
    print(f"Saved FINAL ENSEMBLE PREDICTIONS to: {MODEL_SAVE_NAME}")
    
elif best_individual_rmse < overall_ensemble_rmse and best_individual_rmse < BEST_SINGLE_MLP_RMSE:
    print(f"\nCONCLUSION: A new individual MLP model (Index {best_model_index+1}) is the most accurate predictor.")
    
    final_best_model = models[best_model_index]
    MODEL_SAVE_NAME = f"best_mlp_model_seed_{best_model_seed}.h5" 
    
    # Check if file exists before saving
    if os.path.exists(MODEL_SAVE_NAME):
        print(f"Warning: File {MODEL_SAVE_NAME} already exists. Skipping save to prevent overwrite.")
    else:
        final_best_model.save(MODEL_SAVE_NAME)
        print(f"Saved FINAL BEST INDIVIDUAL MODEL to: {MODEL_SAVE_NAME}")
    
else:
    print("\nCONCLUSION: The original single MLP model remains the most accurate.")

Building and training 9 diverse MLP models...

--- Training MLP Model 1 (Seed: 1) ---
Epoch 1/5000


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 876us/step - loss: 0.0412 - val_loss: 0.0485 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - loss: 0.0168 - val_loss: 0.0272 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - loss: 0.0113 - val_loss: 0.0230 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - loss: 0.0078 - val_loss: 0.0124 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - loss: 0.0054 - val_loss: 0.0110 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.0042 - val_loss: 0.0083 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - loss: 0.0032 - val_loss: 0.0082 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 0.0024 - val_loss: 0.0048 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 0.0020 - val_loss: 0.0036 - learning_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - loss: 0.0476 - val_loss: 0.0428 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - loss: 0.0163 - val_loss: 0.0283 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.0111 - val_loss: 0.0217 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 0.0079 - val_loss: 0.0143 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - loss: 0.0056 - val_loss: 0.0135 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.0042 - val_loss: 0.0082 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.0032 - val_loss: 0.0061 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - loss: 0.0024 - val_loss: 0.0068 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - loss: 0.0023 - val_loss: 0.0055 - learning_

...

503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0480 - val_loss: 0.0334 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - loss: 0.0184 - val_loss: 0.0212 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - loss: 0.0123 - val_loss: 0.0164 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - loss: 0.0084 - val_loss: 0.0127 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0058 - val_loss: 0.0082 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - loss: 0.0043 - val_loss: 0.0092 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - loss: 0.0031 - val_loss: 0.0102 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0025 - val_loss: 0.0039 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - loss: 0.0022 - val_loss: 0.0037 - learning_rate: 

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - loss: 0.0441 - val_loss: 0.0416 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - loss: 0.0163 - val_loss: 0.0289 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 0.0111 - val_loss: 0.0339 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: 0.0076 - val_loss: 0.0201 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - loss: 0.0056 - val_loss: 0.0172 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - loss: 0.0040 - val_loss: 0.0086 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - loss: 0.0032 - val_loss: 0.0081 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - loss: 0.0027 - val_loss: 0.0127 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step - loss: 0.0021 - val_loss: 0.0051 - learning_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 812us/step - loss: 0.0428 - val_loss: 0.0382 - learning_rate: 0.0010
Epoch 2/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 0.0176 - val_loss: 0.0275 - learning_rate: 0.0010
Epoch 3/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - loss: 0.0121 - val_loss: 0.0216 - learning_rate: 0.0010
Epoch 4/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 0.0087 - val_loss: 0.0199 - learning_rate: 0.0010
Epoch 5/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - loss: 0.0061 - val_loss: 0.0143 - learning_rate: 0.0010
Epoch 6/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - loss: 0.0044 - val_loss: 0.0104 - learning_rate: 0.0010
Epoch 7/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 0.0036 - val_loss: 0.0103 - learning_rate: 0.0010
Epoch 8/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 0.0026 - val_loss: 0.0078 - learning_rate: 0.0010
Epoch 9/5000
503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - loss: 0.0022 - val_loss: 0.0093 - learning_